In [19]:
import numpy as np
from scipy import stats
# import cv2
import matplotlib.pyplot as plt
from mpmath import *
from tqdm.auto import tqdm
import os
import time
# Make it so it always prints with decimals and not scientific notation
np.set_printoptions(suppress=True)

GMM Questions

In [2]:
def e_step(x, params):
    """
    Expecation step, takes in x and returns the r
    """
    r = []
    sum_r = 0
    for 𝜆, 𝜇, 𝜎 in params:
        r_ik = 𝜆 * stats.multivariate_normal.pdf(x, mean=𝜇, cov=𝜎, allow_singular=True)
        r.append(r_ik)
        sum_r += r_ik

    return np.array(r) / sum_r

In [3]:
# Test data for E-Step taken from 2017 November Exam
p = np.array([[0.5,250,10],[0.5,20,10]])
x_test = np.array([255,255,1,3,3])
e_step(x_test, p)

array([[1., 1., 0., 0., 0.],
       [0., 0., 1., 1., 1.]])

Expected output:


|n|x|R0|R1|
|---|---|---|---|
|0|255|1|0|
|1|255|1|0|
|2|1|0|1|
|3|3|0|1|
|4|3|0|1|

In [11]:
# Test data taken from 2018 Test 2
p = np.array([[0.25,10,10],[0.75,1,10]])
x_test = np.array([1,2,3,9,7])
np.round(e_step(x_test, p),4)
# Are these correct?

array([[0.0058, 0.0141, 0.0339, 0.8861, 0.5625],
       [0.9942, 0.9859, 0.9661, 0.1139, 0.4375]])

In [18]:
# Test Data from 2019 Test 1
p = np.array([[0.3,15,5 ],[0.7,0,5]])
x_test = np.array([-2,5,7,14,15])
np.round(e_step(x_test, p),4)
# This is slightly wrong again???

array([[0.    , 0.0002, 0.0873, 1.    , 1.    ],
       [1.    , 0.9998, 0.9127, 0.    , 0.    ]])

In [22]:
# Test Data from 2018 Examy
p = np.array([[0.3,50,10],[0.7,150,10]])
x_test = np.array([0,0,200,205,210])
np.round(e_step(x_test, p),4)
# This is slightly wrong again???

array([[1., 1., 0., 0., 0.],
       [0., 0., 1., 1., 1.]])


|n|x|R0|R1|
|---|---|---|---|
|0|0|1|0|
|1|0|1|0|
|2|200|0|1|
|3|205|0|1|
|4|210|0|1|

In [5]:
def m_step(x, r, parameters, two_dim = True):
    """
    Maximisation Step, takes in x and r and maximises the parameters
    """
    new_parameters = []
    for i, (𝜆, 𝜇, 𝜎) in enumerate(parameters):
        ri_sum = np.sum(r[i])
        r_sum = np.sum(r)
        𝜆  = ri_sum / r_sum
        if two_dim:
            𝜇 = np.sum(r[i] * x, axis=1)/ri_sum
            diff = x.T - 𝜇
            cov_i =np.dot(diff.T, np.multiply(r[i].reshape(len(r[i]),1), diff))
            𝜎 = ( cov_i / r[i].sum()) + 1e-4*np.eye(cov_i.shape[0])
        
        else:
            𝜇 = np.sum(r[i]*x)/np.sum(r[i])
            covar_sum = 0
            for j in range(len(x)):
                covar_sum += (r[i][j] * (x[j]-𝜇)*(x[j]-𝜇).T)/np.sum(r[i])
            𝜎 = covar_sum
        new_parameters.append((𝜆, 𝜇, 𝜎))
    return new_parameters

In [9]:
# Test data for M-step taken from Test 2 2017
x_ri = np.array([0, 0.1, 1.1, 1.2, 0.3])
ri_0 = np.array([0.606, 0.593, 0.402, 0.378, 0.563])
ri_1 = np.array([0.394, 0.407, 0.598, 0.622, 0.437])
ris = np.array([ri_0, ri_1])
p = np.array([[0.5, 0.5, 0.4], [0.5, 0.75, 0.3]])
new_p = m_step(x_ri,ris, p, two_dim = False)
print("𝜆",new_p[0][0],new_p[1][0])
print("𝜇",new_p[0][1],new_p[1][1])
print("𝜎",new_p[0][2],new_p[1][2])

𝜆 0.5084 0.4915999999999999
𝜇 0.4421715184893785 0.6411716842961759
𝜎 0.23223415154128194 0.26532686149276935


Correct output:


|#|𝜆|𝜇|𝜎|
|---|---|---|---|
|0|0.5084|0.44217152|0.23223415|
|1|0.4916|0.64117168|0.26532686|


In [12]:
# Test data for M-step taken from Test 2 2018
x_ri = np.array([0, 2, 3, 9, 7])
ri_0 = np.array([0.321, 0.300, 0.279, 0.175, 0.206])
ri_1 = np.array([0.679, 0.700, 0.721, 0.825, 0.794])
ris = np.array([ri_0, ri_1])
p = np.array([[0.25, 10, 10], [0.75, 1, 10]])
new_p = m_step(x_ri,ris, p, two_dim = False)
print("𝜆",new_p[0][0],new_p[1][0])
print("𝜇",new_p[0][1],new_p[1][1])
print("𝜎",new_p[0][2],new_p[1][2])

𝜆 0.2562 0.7438
𝜇 3.4769711163153785 4.4490454423232055
𝜎 9.7529825510783 11.133663380295847


In [24]:
# Test data for M-step taken from Test-1 2019
x_ri = np.array([0,0,200,205,210])
ri_0 = np.array([1, 1, 0, 0, 0])
ri_1 = np.array([0, 0, 1, 1, 1])
ris = np.array([ri_0, ri_1])
p = np.array([[0.3, 50, 10], [0.7, 150, 10]])
new_p = m_step(x_ri,ris, p, two_dim = False)
print("𝜆",new_p[0][0],new_p[1][0])
print("𝜇",new_p[0][1],new_p[1][1])
print("𝜎",new_p[0][2],new_p[1][2])


𝜆 0.4 0.6
𝜇 0.0 205.0
𝜎 0.0 16.666666666666668


Correct output:


|#|𝜆|𝜇|𝜎|
|---|---|---|---|
|0|0.4|0|0|
|1|0.6|205|16.666|
